#Variational Autoencoder with MNIST

##Setup

VAE with Gaussian Latent Variables and Gaussian Posterior distribution.

Specifications:

* Latent variable $\textbf{z} \sim p (\textbf{z}) = \mathcal N_p ({\bf\mu}, {\bf \Sigma})$, with mean zero and ${\bf \Sigma}$ diagonal
* Decoder $p(\mathbf{x}|\mathbf{z})$ is given by a MLP with a single hidden layer
* Encoder $q_\phi({\bf z}|{\bf x})= \mathcal{N}({\bf z}, \boldsymbol{\mu}({\bf x}), \mathrm{diag}(\boldsymbol{\sigma}^2({\bf x})))$
* Cost-function consisting of a reconstruction error with a regularization, that minimizes the KL-Divergence. The reconstruction error is the crossentropy between samples and their reconstructions.
* KL-Divergence for this setup
$$-D_{KL}(q_\phi({\bf z}|{\bf x})|p({\bf z}))={1 \over 2} \sum_{j=1}^J \left (1+\log{\sigma_j^2({\bf x})}-\mu_j^2({\bf x}) -\sigma_j^2({\bf x})\right).
$$

##Import libraries and data

In [0]:
from variational_autoencoder import VariationalAutoencoder
from variational_autoencoder import SampleGenerator
from variational_autoencoder import HiddenVariablePlotter
from keras.datasets import mnist

# Parameters
shape_parameters = {'input_dim': 784,
              'hidden_dim': 256,
              'latent_dim': 2}
fit_parameters = {'optimizer': rmsprop, 
                  'loss': neg_log_likelihood, 
                  'batch_size': 100,
                  'epochs': 3,
                  'epsilon_std': 1.0}

## Load MNIST Dataset

In [0]:
# Data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, input_dim) / 255.
x_test = x_test.reshape(-1, input_dim) / 255.

## Initialize and train VAE

In [0]:
# Distributions
hidden_prior_dist = 'gaussian'          # p(z):       prior distribution of the hidden variables 
encoder_approx = 'gaussian_diagonal'    # q_phi(z|x): mean field approximating intractable p(z|x)
                                        # (further possible approximations: Mises-Fisher, Gaussian Mixtures)

dists = (hidden_prior_dist, encoder_approx)

vae = VariationalAutoencoder(*dists, **shape_parameters)

vae.fit(x_train, x_test, **fit_parameters)

## Plot Samples

In [0]:
n = 5
digit_size = 28
plot_size = (10, 10)

plot = HiddenVariablePlotter(n, digit_size, plot_size)
plot.show()

## Plot embedding in hidden variable space